In [1]:
from pybliometrics.scopus import AuthorSearch
from pybliometrics.scopus import AuthorRetrieval
from scholarly import scholarly
import pandas as pd
import sys
sys.path.append("..")
from rcn_py import orcid, data_storage, neo4j_rsd, openalex



In [3]:
fullname = "Peter Kalverla"
orcid_id = "0000-0002-5025-7862"

author_last = "Kalverla"
author_first = "Peter"

### Scopus

In [ ]:
s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
author_id = s.authors[0].eid.split('-')[-1]

In [ ]:
au = AuthorRetrieval(author_id)
docs = pd.DataFrame(au.get_documents())

### Google Scholar

In [ ]:
# search publication info using title
search_pub = scholarly.search_pubs('17B. 6 LONG TERM OBSERVATIONS OF LONG WAVE RADIATIVE FLUX DIVERGENCE IN THE STABLE BOUNDARY LAYER OVER LAND')
pubinfo=next(search_pub)

In [ ]:
# author info searched by scholar id
search_query_author = scholarly.search_author_id("TOokPzcAAAAJ")
search_query_author

In [ ]:
# author info using name
search_query_author2 = scholarly.search_author('Peter Kalverla')
authorinfo = scholarly.fill(next(search_query_author2))

In [3]:
# publication info
data_storage.scholar_author_info(fullname)

,title,authors,abstarct
0,Low-level jets over the North Sea based on ERA...,"[PC Kalverla, JB Duncan Jr, GJ Steeneveld]",Ten years of ERA5 reanalysis data are combined...
1,An observational climatology of anomalous wind...,"[PC Kalverla, GJ Steeneveld, RJ Ronda]",Uncertainty reduction in offshore wind systems...
2,Quality of wind characteristics in recent wind...,"[PC Kalverla, AAM Holtslag, RJ Ronda]",Offshore wind energy production is rapidly gro...
3,Evaluation of the Weather Research and Forecas...,"[PC Kalverla, GJ Duine]",of down-valley winds in stable stratification ...
4,Evaluation of three mainstream numerical weath...,"[P Kalverla, GJ Steeneveld, R Ronda]",Numerical weather prediction models play an im...
5,A maximum entropy approach to the parametrizat...,"[WTM Verkley, PC Kalverla]",In numerical models of geophysical fluid syste...
6,Clustering wind profile shapes to estimate air...,"[M Schelbergen, PC Kalverla, R Schmehl]",Airborne wind energy (AWE) systems harness ene...
7,Evaluation of the Weather Research and Forecas...,[P Kalverla],"In this study, the representation of the stabl..."
8,Quality of wind characteristics in recent wind...,"[PC Kalverla, AAM Holtslag, RJ Ronda, GJ Steen...",
9,Earth System model evaluation tool (ESMValTool...,"[M Righi, B Andela, V Eyring, A Lauer]",This paper describes the second major release ...


### Crossref (ORCID)

In [13]:
# Using rcn_py.orcid functions
orcid_id = orcid.name_to_orcid_id(fullname)
orcid_record = orcid.query_orcid_for_record(orcid_id)
docs = orcid.extract_works_section(orcid_record)

In [14]:
docs[0]

{'last-modified-date': {'value': 1653871432941},
 'external-ids': {'external-id': [{'external-id-type': 'doi',
    'external-id-value': '10.5194/egusphere-egu2020-11730',
    'external-id-normalized': {'value': '10.5194/egusphere-egu2020-11730',
     'transient': True},
    'external-id-normalized-error': None,
    'external-id-url': {'value': 'https://doi.org/10.5194/egusphere-egu2020-11730'},
    'external-id-relationship': 'self'}]},
 'work-summary': [{'put-code': 70327533,
   'created-date': {'value': 1583795690725},
   'last-modified-date': {'value': 1653871432941},
   'source': {'source-orcid': None,
    'source-client-id': {'uri': 'https://orcid.org/client/0000-0001-9884-1913',
     'path': '0000-0001-9884-1913',
     'host': 'orcid.org'},
    'source-name': {'value': 'Crossref'},
    'assertion-origin-orcid': None,
    'assertion-origin-client-id': None,
    'assertion-origin-name': None},
   'title': {'title': {'value': 'Coupling Hydrological models using BMI in eWaterCycle'},

In [5]:
# All publication info of one researcher
data_storage.crossref_author_by_name(fullname)

,title,doi,authors,abstarct,author_orcid
0,A multi-model ensemble weighting method (ClimW...,10.5194/egusphere-egu21-9387,"[Ruth Lorenz, Lukas Brunner, Peter Kalverla, S...",<jats:p>&amp;lt;p&amp;gt;Too often model evalu...,"[0000-0002-3986-1268, 0000-0001-5760-4524, 000..."
1,Preprocessing of hydrological models&#8217; in...,10.5194/egusphere-egu21-6051,"[Fakhereh Alidoost, Jerom Aerts, Bouwe Andela,...",<jats:p>&amp;lt;p&amp;gt;Hydrological models e...,"[0000-0001-8407-6472, 0000-0003-0157-4818, 000..."
2,Towards Open and FAIR Hydrological Modelling w...,10.5194/egusphere-egu21-7797,"[Niels Drost, Jerom P.M. Aerts, Fakhereh Alido...",<jats:p>&amp;lt;p&amp;gt;The eWaterCycle platf...,"[0000-0001-9795-7981, 0000-0003-0157-4818, 000..."
3,Bringing ESMValTool to the Jupyter Lab,10.5194/egusphere-egu21-4805,"[Peter C. Kalverla, Stef Smeets, Niels Drost, ...",<jats:p>&amp;lt;p&amp;gt;Ease of use can easil...,"[0000-0002-5025-7862, 0000-0002-5413-9038, 000..."
4,Recent developments on the Earth System Model ...,10.5194/egusphere-egu21-3476,"[Bouwe Andela, Fakhereh Alidoost, Lukas Brunne...",<jats:p>&amp;lt;p&amp;gt;The Earth System Mode...,"[0000-0001-9005-8940, 0000-0001-8407-6472, 000..."
5,era5cli: the command line interface to ERA5 data,10.5194/egusphere-egu21-4895,"[Stef Smeets, Jaro Camphuijsen, Niels Drost, F...",<jats:p>&amp;lt;p&amp;gt;With the release of t...,"[0000-0002-5413-9038, 0000-0002-8928-7831, 000..."
6,Clustering wind profile shapes to estimate air...,10.5194/wes-5-1097-2020,"[Mark Schelbergen, Peter C. Kalverla, Roland S...",<jats:p>Abstract. Airborne wind energy (AWE) s...,"[0000-0002-6373-0989, 0000-0002-5025-7862, 000..."
7,era5cli: The command line tool to download ERA...,10.5194/egusphere-egu2020-21619,"[Jaro Camphuijsen, Ronald van Haren, Yifat Dzi...",<jats:p>\n &amp;lt;p&amp;gt;With the re...,"[0000-0002-8928-7831, 0000-0002-3160-3547, 000..."
8,Characterisation of offshore winds for energy ...,10.18174/498797,[Peter C. Kalverla],,[0000-0002-5025-7862]
9,Low-level jets over the North Sea based on ERA...,10.5194/wes-4-193-2019,"[Peter C. Kalverla, James B. Duncan Jr., Gert-...",<jats:p>Abstract. Ten years of ERA5 reanalysis...,"[0000-0002-5025-7862, 0000-0002-8882-5693, 000..."


In [4]:
data_storage.crossref_author_by_orcid(orcid)

,title,doi,authors,abstarct,author_orcid
0,A multi-model ensemble weighting method (ClimW...,10.5194/egusphere-egu21-9387,"[Ruth Lorenz, Lukas Brunner, Peter Kalverla, S...",<jats:p>&amp;lt;p&amp;gt;Too often model evalu...,"[0000-0002-3986-1268, 0000-0001-5760-4524, 000..."
1,Preprocessing of hydrological models&#8217; in...,10.5194/egusphere-egu21-6051,"[Fakhereh Alidoost, Jerom Aerts, Bouwe Andela,...",<jats:p>&amp;lt;p&amp;gt;Hydrological models e...,"[0000-0001-8407-6472, 0000-0003-0157-4818, 000..."
2,Towards Open and FAIR Hydrological Modelling w...,10.5194/egusphere-egu21-7797,"[Niels Drost, Jerom P.M. Aerts, Fakhereh Alido...",<jats:p>&amp;lt;p&amp;gt;The eWaterCycle platf...,"[0000-0001-9795-7981, 0000-0003-0157-4818, 000..."
3,Bringing ESMValTool to the Jupyter Lab,10.5194/egusphere-egu21-4805,"[Peter C. Kalverla, Stef Smeets, Niels Drost, ...",<jats:p>&amp;lt;p&amp;gt;Ease of use can easil...,"[0000-0002-5025-7862, 0000-0002-5413-9038, 000..."
4,Recent developments on the Earth System Model ...,10.5194/egusphere-egu21-3476,"[Bouwe Andela, Fakhereh Alidoost, Lukas Brunne...",<jats:p>&amp;lt;p&amp;gt;The Earth System Mode...,"[0000-0001-9005-8940, 0000-0001-8407-6472, 000..."
5,era5cli: the command line interface to ERA5 data,10.5194/egusphere-egu21-4895,"[Stef Smeets, Jaro Camphuijsen, Niels Drost, F...",<jats:p>&amp;lt;p&amp;gt;With the release of t...,"[0000-0002-5413-9038, 0000-0002-8928-7831, 000..."
6,Clustering wind profile shapes to estimate air...,10.5194/wes-5-1097-2020,"[Mark Schelbergen, Peter C. Kalverla, Roland S...",<jats:p>Abstract. Airborne wind energy (AWE) s...,"[0000-0002-6373-0989, 0000-0002-5025-7862, 000..."
7,era5cli: The command line tool to download ERA...,10.5194/egusphere-egu2020-21619,"[Jaro Camphuijsen, Ronald van Haren, Yifat Dzi...",<jats:p>\n &amp;lt;p&amp;gt;With the re...,"[0000-0002-8928-7831, 0000-0002-3160-3547, 000..."
8,Characterisation of offshore winds for energy ...,10.18174/498797,[Peter C. Kalverla],,[0000-0002-5025-7862]
9,Low-level jets over the North Sea based on ERA...,10.5194/wes-4-193-2019,"[Peter C. Kalverla, James B. Duncan Jr., Gert-...",<jats:p>Abstract. Ten years of ERA5 reanalysis...,"[0000-0002-5025-7862, 0000-0002-8882-5693, 000..."


In [4]:
# Get all publications of all coauthors and do de-duplication
# Long running time
df = data_storage.coauthor_data_crossref(orcid_id)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
node_data = data_storage.assign_group_node(orcid_id)

In [ ]:
links = data_storage.get_links_from_coauthor_rel(orcid_id)

In [ ]:
data_storage.build_networkx(orcid_id)

In [2]:
data_storage.build_networkx('0000-0002-5025-7862')

ValueError: cannot compute LDA over an empty collection (no terms)

### RSD

In [ ]:
rsd_data = neo4j_rsd.request_rsd_data()

### OpenAlex

In [ ]:
author_result_1 = openalex.search_user_by_orcid(orcid_id)
author_result_2 = openalex.search_user_by_name(fullname)


In [ ]:
works_1 = openalex.search_works_by_orcid(orcid_id)
works_2 = openalex.search_works_by_name(fullname)


In [ ]:
doi = "https://doi.org/10.7717/peerj.4375"
publication_result = openalex.get_pub_info_by_doi(doi)      # all the information of this publication
keywords = openalex.publication_keywords(doi)               # for topic modeling

In [ ]:
institution_name = "Netherlands eScience Center"
institution = openalex.find_institution(institution_name)

inst_works_1 = openalex.get_some_works_of_one_institution(institution_name)     # Fast, only return 25 work results
inst_works_2 = openalex.get_works_of_one_institution(institution_name, 100)     # Set work count
inst_works_3 = openalex.get_all_works_of_one_institution(institution_name)      # All works
inst_works_4 = openalex.get_works_of_one_institution_by_year(institution_name, 2021, 2023) # Year period 

In [ ]:
colla_institutions = openalex.coauthor_institutions(institution_name)               # Collaboration between institutions
node_list, coauthor_pair = openalex.build_institution_network(institution_name)     # Create nodes and relations of the network